In [1]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
import optuna

In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [1149, 4836, 9443, 7483, 3492, 6287,  830, 3294, 5932,  815, 7070,
       1529, 3454, 6073, 7831, 1069, 3632,  953,  368, 3999, 9981,  906,
       6792, 1439, 8439, 9240, 8162,   32, 5785, 1013, 2747, 2702, 4807,
       831, 3985, 4003, 4758, 7955, 3976, 5436, 8634, 2511, 8627, 6888,
       4179, 1565, 9181, 6389, 5879, 1216, 8749, 8996,   99, 1481, 4120,
       8561, 7008, 5624,  271, 5204, 9169, 2537, 4070, 5458, 7383,  137,
       8414, 6327, 3119, 5659, 2907, 9390, 7055, 9770, 9614, 6769,  694,
       7854, 9925, 7895, 5259, 9598, 6352, 7884, 5592, 7017, 7374, 2208,
       7364, 1995, 7264, 2520, 3922, 7704, 2333, 1601, 2534, 6556, 9072,
       3046]

In [3]:
data = pl.scan_parquet(dataset_path + dataset_file).filter(pl.col("foto_mes") == 202108).collect()

In [4]:
# convirto a pandas dataframe
data = data.to_pandas()

In [5]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [6]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [7]:
X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)

### Predicciones para Kaggle

In [8]:
# Voy a probar 20 puntos de corte para cada semilla (total 100 envios)

version = 'v016' # UPADTE

modelos = [f'lgb_competencia2_{version}_s{semilla}_final.txt' for semilla in semillas]

In [9]:
len(modelos)

100

In [10]:
modelos_path

'C:/Eugenio/Maestria/DMEyF/modelos/'

In [11]:
X_test_with_predictions = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()

In [12]:
counter = 0

for modelo in modelos:
    
    # cargamos el modelo
    model = lgb.Booster(model_file=f'{modelos_path}{version}/{modelo}')
    
    # predecimos para agosto
    X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()
    y_pred_lgm = model.predict(X_test)
    X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm
    
    print(f'Predicciones del modelo: {modelo} DONE')
    
    counter += 1

Predicciones del modelo: lgb_competencia2_v016_s1149_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s4836_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s9443_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s7483_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s3492_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s6287_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s830_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s3294_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s5932_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s815_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s7070_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s1529_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s3454_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s6073_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s78

C:\Users\eaneg\AppData\Local\Temp\ipykernel_20672\3572328474.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm


Predicciones del modelo: lgb_competencia2_v016_s9072_final.txt DONE
Predicciones del modelo: lgb_competencia2_v016_s3046_final.txt DONE


C:\Users\eaneg\AppData\Local\Temp\ipykernel_20672\3572328474.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm


In [13]:
X_test_with_predictions = X_test_with_predictions.copy()

In [14]:
X_test_with_predictions['predicted_avg'] = X_test_with_predictions.iloc[:, -100:].mean(axis=1)

In [15]:
# ordeno de mayor probabilidad de baja a menor
idx = np.argsort(X_test_with_predictions['predicted_avg'])[::-1]
X_test_with_predictions.reset_index(drop=True, inplace=True)
X_test_with_predictions = X_test_with_predictions.iloc[idx]

In [16]:
# aux = X_test_with_predictions.head(100)
# aux

In [17]:
puntos_corte = np.linspace(9000, 13000, 50).astype(int)
puntos_corte

array([ 9000,  9081,  9163,  9244,  9326,  9408,  9489,  9571,  9653,
        9734,  9816,  9897,  9979, 10061, 10142, 10224, 10306, 10387,
       10469, 10551, 10632, 10714, 10795, 10877, 10959, 11040, 11122,
       11204, 11285, 11367, 11448, 11530, 11612, 11693, 11775, 11857,
       11938, 12020, 12102, 12183, 12265, 12346, 12428, 12510, 12591,
       12673, 12755, 12836, 12918, 13000])

In [18]:
counter = 0

for corte in puntos_corte:
    
    envios = np.zeros(len(X_test_with_predictions), dtype=int)
    envios[:corte] = 1
    X_test_with_predictions['Predicted'] = envios
    
    output = X_test_with_predictions[['numero_de_cliente', 'Predicted']]
        
    counter += 1
    
    file_name = f'results_{version}_{counter}.csv'
    output_path = base_path + f'exp/competencia_2/{version}/' + file_name
    
    output.to_csv(output_path, index=False)
            
    print(f'{counter}. corte: {corte}')

1. corte: 9000
2. corte: 9081
3. corte: 9163
4. corte: 9244
5. corte: 9326
6. corte: 9408
7. corte: 9489
8. corte: 9571
9. corte: 9653
10. corte: 9734
11. corte: 9816
12. corte: 9897
13. corte: 9979
14. corte: 10061
15. corte: 10142
16. corte: 10224
17. corte: 10306
18. corte: 10387
19. corte: 10469
20. corte: 10551
21. corte: 10632
22. corte: 10714
23. corte: 10795
24. corte: 10877
25. corte: 10959
26. corte: 11040
27. corte: 11122
28. corte: 11204
29. corte: 11285
30. corte: 11367
31. corte: 11448
32. corte: 11530
33. corte: 11612
34. corte: 11693
35. corte: 11775
36. corte: 11857
37. corte: 11938
38. corte: 12020
39. corte: 12102
40. corte: 12183
41. corte: 12265
42. corte: 12346
43. corte: 12428
44. corte: 12510
45. corte: 12591
46. corte: 12673
47. corte: 12755
48. corte: 12836
49. corte: 12918
50. corte: 13000
